In [1]:
#import required packages
import boto3
import requests
import json
import pandas as pd
from datetime import datetime
import time
from pyathena import connect
import sagemaker
from sqlalchemy.engine import create_engine



/opt/conda/lib/python3.11/site-packages/pydantic/_internal/_fields.py:192: UserWarning: Field name "json" in "MonitoringDatasetFormat" shadows an attribute in parent "Base"
  warnings.warn(


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


## Check s3 for files

In [2]:
# Create an S3 client
s3 = boto3.client('s3')

# List objects with the prefix
response = s3.list_objects_v2(Bucket='ecogridaidata', Prefix='eia_electricity/')

# Print the file keys
for obj in response.get('Contents', []):
    print(obj['Key'])

eia_electricity/cleaned/
eia_electricity/cleaned/output/forecasting-deepar-2025-04-01-04-15-42-909/debug-output/training_job_end.ts
eia_electricity/cleaned/output/forecasting-deepar-2025-04-01-04-15-42-909/profiler-output/framework/training_job_end.ts
eia_electricity/cleaned/output/forecasting-deepar-2025-04-01-04-15-42-909/profiler-output/system/incremental/2025040104/1743480960.algo-1.json
eia_electricity/cleaned/output/forecasting-deepar-2025-04-01-04-15-42-909/profiler-output/system/incremental/2025040104/1743481020.algo-1.json
eia_electricity/cleaned/output/forecasting-deepar-2025-04-01-04-15-42-909/profiler-output/system/incremental/2025040104/1743481080.algo-1.json
eia_electricity/cleaned/output/forecasting-deepar-2025-04-01-04-15-42-909/profiler-output/system/incremental/2025040104/1743481140.algo-1.json
eia_electricity/cleaned/output/forecasting-deepar-2025-04-01-04-15-42-909/profiler-output/system/training_job_end.ts
eia_electricity/cleaned/output/forecasting-deepar-2025-04-0

Note: If data is not available, locate "RTC:ADS508TeamProject/installation/Installation & Set-Up.ipynb" notebook to run refesh data from the EIA website

## Load data stored as a json file in s3 "ecogridaidata" into a dataframe for verification and processing

In [3]:
# S3 path to your file
s3_uri = "s3://ecogridaidata/eia_electricity/eia_data_subregion_20250323_191441.json"
s3_uri2 = "s3://ecogridaidata/eia_electricity/eia_data_energy_source_20250323_200432.json"
s3_uri3 = "s3://ecogridaidata/eia_electricity/eia_data_neighboring_bal_20250324_224805.json"
s3_uri4 = "s3://ecogridaidata/eia_electricity/eia_data_demand_20250325_021240.json"

# Read json directly into pandas DataFrame
df_subregion = pd.read_json(s3_uri, storage_options={"anon": False})
df_energysource = pd.read_json(s3_uri2, storage_options={"anon": False})
df_neighbor_bal = pd.read_json(s3_uri3, storage_options={"anon": False})
df_demand = pd.read_json(s3_uri4, storage_options={"anon": False})

# Preview the data
df_subregion.head()

,period,subba,subba-name,parent,parent-name,timezone,value,value-units
0,2024-12-31,PGAE,Pacific Gas and Electric,CISO,California Independent System Operator,Arizona,247213,megawatthours
1,2024-12-31,PGAE,Pacific Gas and Electric,CISO,California Independent System Operator,Central,247876,megawatthours
2,2024-12-31,PGAE,Pacific Gas and Electric,CISO,California Independent System Operator,Eastern,248481,megawatthours
3,2024-12-31,PGAE,Pacific Gas and Electric,CISO,California Independent System Operator,Mountain,247213,megawatthours
4,2024-12-31,PGAE,Pacific Gas and Electric,CISO,California Independent System Operator,Pacific,246697,megawatthours


In [4]:
df_energysource.head()

,period,respondent,respondent-name,fueltype,type-name,timezone,timezone-description,value,value-units
0,2024-12-31,AECI,"Associated Electric Cooperative, Inc.",COL,Coal,Arizona,Arizona,23782,megawatthours
1,2024-12-31,AECI,"Associated Electric Cooperative, Inc.",COL,Coal,Central,Central,23309,megawatthours
2,2024-12-31,AECI,"Associated Electric Cooperative, Inc.",COL,Coal,Eastern,Eastern,22893,megawatthours
3,2024-12-31,AECI,"Associated Electric Cooperative, Inc.",COL,Coal,Mountain,Mountain,23782,megawatthours
4,2024-12-31,AECI,"Associated Electric Cooperative, Inc.",COL,Coal,Pacific,Pacific,24422,megawatthours


In [5]:
df_neighbor_bal.head()

,period,fromba,fromba-name,toba,toba-name,timezone,value,value-units
0,2024-12-31,AECI,"Associated Electric Cooperative, Inc.",MISO,"Midcontinent Independent System Operator, Inc.",Arizona,584,megawatthours
1,2024-12-31,AECI,"Associated Electric Cooperative, Inc.",MISO,"Midcontinent Independent System Operator, Inc.",Central,359,megawatthours
2,2024-12-31,AECI,"Associated Electric Cooperative, Inc.",MISO,"Midcontinent Independent System Operator, Inc.",Eastern,321,megawatthours
3,2024-12-31,AECI,"Associated Electric Cooperative, Inc.",MISO,"Midcontinent Independent System Operator, Inc.",Mountain,584,megawatthours
4,2024-12-31,AECI,"Associated Electric Cooperative, Inc.",MISO,"Midcontinent Independent System Operator, Inc.",Pacific,1111,megawatthours


In [6]:
df_demand.head()

,period,respondent,respondent-name,type,type-name,timezone,timezone-description
0,2025-03-24,AECI,"Associated Electric Cooperative, Inc.",DF,Day-ahead demand forecast,Central,Central
1,2025-03-24,AECI,"Associated Electric Cooperative, Inc.",DF,Day-ahead demand forecast,Eastern,Eastern
2,2025-03-24,AVA,Avista Corporation,DF,Day-ahead demand forecast,Arizona,Arizona
3,2025-03-24,AVA,Avista Corporation,DF,Day-ahead demand forecast,Central,Central
4,2025-03-24,AVA,Avista Corporation,DF,Day-ahead demand forecast,Eastern,Eastern


## Create Athena Database

In [7]:
# Save to CSV for Athena readability
df_subregion.to_csv("s3://ecogridaidata/eia_electricity/subregion.csv", index=False)
df_energysource.to_csv("s3://ecogridaidata/eia_electricity/data_csv/energysource.csv", index=False)
df_neighbor_bal.to_csv("s3://ecogridaidata/eia_electricity/data_csv/neighboring_bal.csv", index=False)
df_demand.to_csv("s3://ecogridaidata/eia_electricity/demand.csv", index=False)

In [8]:
# Setup
sess = sagemaker.Session()
bucket = 'ecogridaidata'
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

s3_staging_dir = f"s3://{bucket}/athena/staging"
database_name = "ecodataaidatabase"

# Connect
conn = connect(region_name=region, s3_staging_dir=s3_staging_dir)
cursor = conn.cursor()

# Create DB if needed
cursor.execute("CREATE DATABASE IF NOT EXISTS ecodataaidatabase")

In [9]:
# Subregion Table
query = f"""
CREATE EXTERNAL TABLE IF NOT EXISTS {database_name}.subregion (
    period string,
    subba string,
    `subba-name` string,
    parent string,
    `parent-name` string,
    timezone string,
    value int,
    `value-units` string
)
ROW FORMAT SERDE 'org.apache.hadoop.hive.serde2.OpenCSVSerde'
WITH SERDEPROPERTIES (
  "separatorChar" = ",",
  "quoteChar" = "\\""
)
LOCATION 's3://{bucket}/athena/subregion/'
TBLPROPERTIES ('skip.header.line.count'='1');
"""

cursor = conn.cursor()
cursor.execute(query)

In [10]:
# Test Query Subregion
query = f"SELECT * FROM {database_name}.subregion LIMIT 10;"
df = pd.read_sql(query, conn)
df.head()

/tmp/ipykernel_4101/2186447736.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


,period,subba,subba-name,parent,parent-name,timezone,value,value-units
0,2024-12-31,PGAE,Pacific Gas and Electric,CISO,California Independent System Operator,Arizona,247213,megawatthours
1,2024-12-31,PGAE,Pacific Gas and Electric,CISO,California Independent System Operator,Central,247876,megawatthours
2,2024-12-31,PGAE,Pacific Gas and Electric,CISO,California Independent System Operator,Eastern,248481,megawatthours
3,2024-12-31,PGAE,Pacific Gas and Electric,CISO,California Independent System Operator,Mountain,247213,megawatthours
4,2024-12-31,PGAE,Pacific Gas and Electric,CISO,California Independent System Operator,Pacific,246697,megawatthours


In [11]:
# Energy Source Table
query = """
CREATE EXTERNAL TABLE IF NOT EXISTS ecodataaidatabase.energysource (
    period string,
    respondent string,
    `respondent-name` string,
    fueltype string,
    `type-name` string,
    timezone string,
    `timezone-description` string,
    value int,
    `value-units` string
)
ROW FORMAT SERDE 'org.apache.hadoop.hive.serde2.OpenCSVSerde'
WITH SERDEPROPERTIES (
  "separatorChar" = ",",
  "quoteChar" = "\\""
)
LOCATION 's3://ecogridaidata/athena/energysource/'
TBLPROPERTIES ('skip.header.line.count'='1');
"""
cursor = conn.cursor()
cursor.execute(query)

In [12]:
# Neighboring Table
query = """
CREATE EXTERNAL TABLE IF NOT EXISTS ecodataaidatabase.neighboring_bal (
    period string,
    fromba string,
    `fromba-name` string,
    toba string,
    `toba-name` string,
    timezone string,
    value int,
    `value-units` string
)
ROW FORMAT SERDE 'org.apache.hadoop.hive.serde2.OpenCSVSerde'
WITH SERDEPROPERTIES (
  "separatorChar" = ",",
  "quoteChar" = "\\""
)
LOCATION 's3://ecogridaidata/athena/neighboring_bal/'
TBLPROPERTIES ('skip.header.line.count'='1');
"""
cursor = conn.cursor()
cursor.execute(query)

In [13]:
# Demand Table
query = """
CREATE EXTERNAL TABLE IF NOT EXISTS ecodataaidatabase.demand (
    period string,
    respondent string,
    `respondent-name` string,
    type string,
    `type-name` string,
    timezone string,
    `timezone-description` string,
    value int,
    `value-units` string
)
ROW FORMAT SERDE 'org.apache.hadoop.hive.serde2.OpenCSVSerde'
WITH SERDEPROPERTIES (
  "separatorChar" = ",",
  "quoteChar" = "\\""
)
LOCATION 's3://ecogridaidata/athena/demand/'
TBLPROPERTIES ('skip.header.line.count'='1');
"""
# Run the query
cursor = conn.cursor()
cursor.execute(query)

In [14]:
# Verification
df_tables = pd.read_sql(f"SHOW TABLES IN {database_name}", conn)
print(df_tables)

/tmp/ipykernel_4101/300048193.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_tables = pd.read_sql(f"SHOW TABLES IN {database_name}", conn)


          tab_name
0           demand
1     energysource
2  neighboring_bal
3        subregion


In [15]:
if database_name in df_tables.values:
    ingest_create_athena_db_passed = True

## Exploratory Data Analysis

In [16]:
for name, df in zip(
    ['Subregion', 'Energy Source', 'Neighbor Balancing', 'Demand'],
    [df_subregion, df_energysource, df_neighbor_bal, df_demand]
):
    print(f"\n=== {name} Dataset ===")
    print("Shape:", df.shape)
    print("Null values:\n", df.isnull().sum())
    print("Data types:\n", df.dtypes)


=== Subregion Dataset ===
Shape: (302719, 8)
Null values:
 period         0
subba          0
subba-name     0
parent         0
parent-name    0
timezone       0
value          0
value-units    0
dtype: int64
Data types:
 period         object
subba          object
subba-name     object
parent         object
parent-name    object
timezone       object
value           int64
value-units    object
dtype: object

=== Energy Source Dataset ===
Shape: (1425802, 9)
Null values:
 period                  0
respondent              0
respondent-name         0
fueltype                0
type-name               0
timezone                0
timezone-description    0
value                   0
value-units             0
dtype: int64
Data types:
 period                  object
respondent              object
respondent-name         object
fueltype                object
type-name               object
timezone                object
timezone-description    object
value                    int64
value-units    

Datasets do no contain any null values

In [17]:
dfs = [df_subregion, df_energysource, df_neighbor_bal, df_demand]
for df in dfs:
    df['period'] = pd.to_datetime(df['period'])

for name, df in zip(
    ['Subregion', 'Energy Source', 'Neighbor Balancing', 'Demand'],
    dfs
):
    years = df['period'].dt.year.unique()
    print(f"\n{name} years in data:", sorted(years))


Subregion years in data: [2023, 2024]

Energy Source years in data: [2023, 2024]

Neighbor Balancing years in data: [2023, 2024]

Demand years in data: [2025]


In [18]:
print(df_demand.columns)

Index(['period', 'respondent', 'respondent-name', 'type', 'type-name',
       'timezone', 'timezone-description'],
      dtype='object')


## Data Preparation

In [19]:
# --- Subregion Dataset ---
query_subregion = "SELECT * FROM ecodataaidatabase.subregion"
df_subregion = pd.read_sql(query_subregion, conn)

# Clean: drop unnecessary columns
df_subregion.drop(columns=['subba-name', 'parent-name', 'value-units'], inplace=True)

# Scrub: drop duplicates and rows missing critical fields
df_subregion.drop_duplicates(inplace=True)
df_subregion.dropna(subset=['period', 'value'], inplace=True)

# Feature engineering: convert period to datetime and create new features
df_subregion['period'] = pd.to_datetime(df_subregion['period'])
df_subregion['month'] = df_subregion['period'].dt.month
df_subregion['weekday'] = df_subregion['period'].dt.weekday

# Preview
print("Subregion DataFrame after cleaning:")
display(df_subregion.head())

/tmp/ipykernel_4101/4198048877.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_subregion = pd.read_sql(query_subregion, conn)


Subregion DataFrame after cleaning:


,period,subba,parent,timezone,value,month,weekday
0,2024-12-31,PGAE,CISO,Arizona,247213,12,1
1,2024-12-31,PGAE,CISO,Central,247876,12,1
2,2024-12-31,PGAE,CISO,Eastern,248481,12,1
3,2024-12-31,PGAE,CISO,Mountain,247213,12,1
4,2024-12-31,PGAE,CISO,Pacific,246697,12,1


In [20]:
# --- Energysource Dataset ---
query_energysource = "SELECT * FROM ecodataaidatabase.energysource"
df_energysource = pd.read_sql(query_energysource, conn)

# Clean: drop unnecessary columns
df_energysource.drop(columns=['respondent-name', 'type-name', 'timezone-description', 'value-units'], inplace=True)

# Scrub: drop duplicates and rows missing critical fields
df_energysource.drop_duplicates(inplace=True)
df_energysource.dropna(subset=['period', 'value', 'fueltype'], inplace=True)

# Feature engineering: convert period to datetime and add time features
df_energysource['period'] = pd.to_datetime(df_energysource['period'])
df_energysource['month'] = df_energysource['period'].dt.month
df_energysource['weekday'] = df_energysource['period'].dt.weekday

# Preview
print("Energysource DataFrame after cleaning:")
display(df_energysource.head())

/tmp/ipykernel_4101/2554255695.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_energysource = pd.read_sql(query_energysource, conn)


Energysource DataFrame after cleaning:


,period,respondent,fueltype,timezone,value,month,weekday
0,2023-06-23,TEPC,OTH,Pacific,-8,6,4
1,2023-06-23,TEPC,SUN,Arizona,4184,6,4
2,2023-06-23,TEPC,SUN,Central,4184,6,4
3,2023-06-23,TEPC,SUN,Eastern,4184,6,4
4,2023-06-23,TEPC,SUN,Mountain,4184,6,4


In [21]:
# --- Neighboring Balancing Dataset ---
query_neighboring = "SELECT * FROM ecodataaidatabase.neighboring_bal"
df_neighboring_bal = pd.read_sql(query_neighboring, conn)

# Clean: drop unnecessary columns
df_neighboring_bal.drop(columns=['fromba-name', 'toba-name', 'value-units'], inplace=True)

# Scrub: drop duplicates and rows missing critical fields
df_neighboring_bal.drop_duplicates(inplace=True)
df_neighboring_bal.dropna(subset=['period', 'value', 'fromba', 'toba'], inplace=True)

# Feature engineering: convert period to datetime and add new features
df_neighboring_bal['period'] = pd.to_datetime(df_neighboring_bal['period'])
df_neighboring_bal['month'] = df_neighboring_bal['period'].dt.month
df_neighboring_bal['weekday'] = df_neighboring_bal['period'].dt.weekday

# Preview
print("Neighboring Balancing DataFrame after cleaning:")
display(df_neighboring_bal.head())

/tmp/ipykernel_4101/238914452.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_neighboring_bal = pd.read_sql(query_neighboring, conn)


Neighboring Balancing DataFrame after cleaning:


,period,fromba,toba,timezone,value,month,weekday
0,2024-12-31,AECI,MISO,Arizona,584,12,1
1,2024-12-31,AECI,MISO,Central,359,12,1
2,2024-12-31,AECI,MISO,Eastern,321,12,1
3,2024-12-31,AECI,MISO,Mountain,584,12,1
4,2024-12-31,AECI,MISO,Pacific,1111,12,1


In [22]:
# --- Demand Dataset ---
query_demand = "SELECT * FROM ecodataaidatabase.demand"
df_demand = pd.read_sql(query_demand, conn)

# Scrub: drop duplicates and rows missing critical fields
df_demand.drop_duplicates(inplace=True)
df_demand.dropna(subset=['period', 'value'], inplace=True)

# Feature engineering: convert period to datetime and add new features
df_demand['period'] = pd.to_datetime(df_demand['period'])
df_demand['month'] = df_demand['period'].dt.month
df_demand['weekday'] = df_demand['period'].dt.weekday

# Preview
print("Demand DataFrame after cleaning:")
display(df_demand.head())

/tmp/ipykernel_4101/3728839611.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_demand = pd.read_sql(query_demand, conn)


Demand DataFrame after cleaning:


,period,respondent,respondent-name,type,type-name,timezone,timezone-description,value,value-units,month,weekday


In [23]:
# Test Demand Query
df = pd.read_sql("SELECT * FROM ecodataaidatabase.demand LIMIT 10;", conn)
df.head()

/tmp/ipykernel_4101/2974498777.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql("SELECT * FROM ecodataaidatabase.demand LIMIT 10;", conn)


,period,respondent,respondent-name,type,type-name,timezone,timezone-description,value,value-units
0,2025-03-23,SPA,Southwestern Power Administration,NG,Net generation,Eastern,Eastern,None,None
1,2025-03-23,SPA,Southwestern Power Administration,NG,Net generation,Mountain,Mountain,None,None
2,2025-03-23,SPA,Southwestern Power Administration,NG,Net generation,Pacific,Pacific,None,None
3,2025-03-23,SPA,Southwestern Power Administration,TI,Total interchange,Central,Central,None,None
4,2025-03-23,SPA,Southwestern Power Administration,TI,Total interchange,Eastern,Eastern,None,None


In [24]:
#prepare dataset for forecasting
import pandas as pd
import boto3
import sagemaker
from sagemaker import get_execution_role
from sagemaker.estimator import Estimator
from sagemaker import image_uris
import matplotlib.pyplot as plt

# Step 1: Setup
session = sagemaker.Session()
role = get_execution_role()
bucket = 'ecogridaidata'
prefix = 'eia_electricity/cleaned'
region = boto3.Session().region_name
container = image_uris.retrieve(region=region, framework='forecasting-deepar')



[04/14/25 02:59:09] INFO     Same images used for training and inference. Defaulting to image     ]8;id=28317;file:///opt/conda/lib/python3.11/site-packages/sagemaker/image_uris.py\image_uris.py]8;;\:]8;id=252346;file:///opt/conda/lib/python3.11/site-packages/sagemaker/image_uris.py#393\393]8;;\
                             scope: inference.                                                                     

                    INFO     Ignoring unnecessary instance type: None.                            ]8;id=244569;file:///opt/conda/lib/python3.11/site-packages/sagemaker/image_uris.py\image_uris.py]8;;\:]8;id=190384;file:///opt/conda/lib/python3.11/site-packages/sagemaker/image_uris.py#530\530]8;;\

In [25]:
#Step 2: standardize timestamps
df['period'] = pd.to_datetime(df['period'])

#Step 3: Convert each 'value' series into a list
subregion_series = df_subregion.sort_values('period')['value'].tolist()
energysource_series = df_energysource.sort_values('period')['value'].tolist()
neighbor_series = df_neighbor_bal.sort_values('period')['value'].tolist()
demand_series = df_demand.sort_values('period')['value'].tolist()

In [26]:
#Step 4: Generate DeepAR
deepar_input = {
    "start": str(df_demand['period'].min()),
    "target": demand_series,
    "dynamic_feat": [subregion_series, energysource_series, neighbor_series],
    "item_id": "US_demand_forecast"
}

In [27]:
#Save and uploads to bucket
import json
s3_key = f'{prefix}/train/train.json'
s3_path = f's3://{bucket}/{s3_key}'
json_data = json.dumps(deepar_input)

s3 = boto3.client('s3')
s3.put_object(Bucket=bucket, Key=s3_key, Body=json_data)

{'ResponseMetadata': {'RequestId': 'D7VFRM8YCA1VTDN2',
  'HostId': 'KpJ4zWOJ6vrkb1iYdEvM4XcGNVuHvxBRgZRjs4GUSffwul0u1WPSrE70ETebeDcA3OnyuDX7bKI=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'KpJ4zWOJ6vrkb1iYdEvM4XcGNVuHvxBRgZRjs4GUSffwul0u1WPSrE70ETebeDcA3OnyuDX7bKI=',
   'x-amz-request-id': 'D7VFRM8YCA1VTDN2',
   'date': 'Mon, 14 Apr 2025 03:02:17 GMT',
   'x-amz-server-side-encryption': 'AES256',
   'etag': '"b919382d8fc2afadd3cbe03dde0e5ca6"',
   'x-amz-checksum-crc32': 'Vv8LWQ==',
   'x-amz-checksum-type': 'FULL_OBJECT',
   'content-length': '0',
   'server': 'AmazonS3'},
  'RetryAttempts': 0},
 'ETag': '"b919382d8fc2afadd3cbe03dde0e5ca6"',
 'ChecksumCRC32': 'Vv8LWQ==',
 'ChecksumType': 'FULL_OBJECT',
 'ServerSideEncryption': 'AES256'}